# Trying to check if the python script can guess what the person is speaking in the audio

In [2]:
# Script for the same

import pydub
from pydub import AudioSegment
import speech_recognition as sr

# Load the MP3 file
audio = AudioSegment.from_file("latest_input.wav")

# Convert the audio to WAV format (required by the speech recognition library)
wav_audio = audio.set_frame_rate(16000).set_channels(1)
wav_audio.export("output.wav", format="wav")

# Use the speech recognition library to recognize the speech
r = sr.Recognizer()
with sr.AudioFile("output.wav") as source:
    audio_data = r.record(source)
    text = r.recognize_google(audio_data)

# Print the recognized text
print(text)

this place be going down when you hear that 3 + 6 sound


In [3]:
# Analysed the audio by slowing down to each bit and seems that the audio is pretty accurately determined specially at the " 3 + 6 " part
# Where there might be a confusion for "Triple Six" to be said.

In [6]:
import librosa
import numpy as np

# Load the WAV file
y, sr = librosa.load("latest_input.wav")

# Perform onset detection to identify the beat locations
onset_envelope = librosa.onset.onset_strength(y=y, sr=sr)
onset_frames = librosa.onset.onset_detect(onset_envelope=onset_envelope, sr=sr)

# Convert the onset frames to time values
onset_times = librosa.frames_to_time(onset_frames, sr=sr)

# Create a new beat track by interpolating the onset times
beat_times = np.linspace(onset_times[0], onset_times[-1], num=int(sr * 5))

# Export the beat track as a new audio file
beat_audio = librosa.beats_to_audio(beat_times, sr=sr)
librosa.output.write_wav("output.wav", beat_audio, sr)

C:\Users\aryan\AppData\Local\Temp\ipykernel_17660\1608227863.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load("latest_input.wav")


AttributeError: No librosa attribute beats_to_audio

In [11]:
import librosa
import numpy as np

# Load the WAV file
y, sr = librosa.load("latest_input.wav")

# Perform onset detection to identify the beat locations
onset_envelope = librosa.onset.onset_strength(y=y, sr=sr)
onset_frames = librosa.onset.onset_detect(onset_envelope=onset_envelope, sr=sr)

# Convert the onset frames to time values
onset_times = librosa.frames_to_time(onset_frames, sr=sr)

# Create a new beat track by interpolating the onset times
beat_times = np.linspace(onset_times[0], onset_times[-1], num=int(sr * 5))

# Convert beat times to a time-domain signal
beat_audio = librosa.istft(beat_times, hop_length=512, win_length=1024)

# Export the beat track as a new audio file
librosa.output.write_wav("output.wav", beat_audio, sr)

C:\Users\aryan\AppData\Local\Temp\ipykernel_17660\3670970376.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load("latest_input.wav")


IndexError: tuple index out of range

In [2]:
#Poojan latest
import numpy as np
import librosa
import soundfile as sf
import audioread

def simple_delay_reverb(audio, sr, delay_ms=300, decay=0, repeats=2):
    delay_samples = int(sr * delay_ms / 1000)
    output = np.copy(audio)
    for _ in range(repeats):
        delayed = np.zeros_like(output)
        delayed[delay_samples:] = output[:-delay_samples]
        output += delayed * decay
    output = output / np.max(np.abs(output))
    return output

def load_audio_with_fallback(file_path, sr=None):
    try:
        return librosa.load(file_path, sr=sr)
    except Exception as e:
        print(f"Failed to load with librosa due to: {e}. Falling back to audioread.")
        with audioread.audio_open(file_path) as f:
            data = np.hstack([np.frombuffer(chunk, np.int16) for chunk in f])
            if sr is not None:
                data = librosa.resample(data.astype(float), f.samplerate, sr)
            return data, sr

def modify_vocals(vocals_filename, beat_filename, output_filename, volume_gain=2.0, beat_start_time=0.0, pitch_shift_semitones=0):
    vocals, sr_vocals = load_audio_with_fallback(vocals_filename, sr=None)
    beat, sr_beat = load_audio_with_fallback(beat_filename, sr=sr_vocals)

    if pitch_shift_semitones != 0:
        vocals = librosa.effects.pitch_shift(vocals, sr_vocals, pitch_shift_semitones)

    vocals = simple_delay_reverb(vocals, sr_vocals)

    start_sample = int(beat_start_time * sr_beat)
    if start_sample < len(beat):
        beat = beat[start_sample:]
    else:
        raise ValueError("Beat start time exceeds the length of the beat track.")

    vocals = vocals * volume_gain

    min_length = min(len(vocals), len(beat))
    vocals = vocals[:min_length]
    beat = beat[:min_length]

    mixed = vocals + beat
    mixed = mixed / np.max(np.abs(mixed))

    sf.write(output_filename, mixed, sr_vocals)

def main():
    vocals_file = r"latest_input.wav"
    beat_file = r"Beat.wav"
    output_file = r"Final_P_O_3.wav"

    modify_vocals(vocals_file, beat_file, output_file, volume_gain=1.0, beat_start_time=3.2, pitch_shift_semitones=0)

if __name__ == "__main__":
    main()